In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lib.actor.LeeNet import LeeNetActor
from lib.models.backbone.pure_RMT import PureRMT
from lib.models.LeeNet.score_pureRMT_mlp import ScorePureRMTMLP
from lib.models.head.mlp import MLP
from lib.trainer.LeeNet_trainer import LeeNetTrainer
from lib.utils.base_funtion import build_dataloaders, get_optimizer_scheduler
from lib.config.cfg_loader import env_setting
from torch.nn.functional import l1_loss
from torch.nn import BCEWithLogitsLoss
from lib.utils.box_ops import giou_loss
from lib.utils.focal_loss import FocalLoss
import torch


def build_model(cfg):
    backbone = PureRMT(cfg=cfg)
    head = MLP(input_dim=10 * cfg.model.pureRMT.embed_dim[-1], hidden_dim=cfg.model.pureRMT.embed_dim[-1], output_dim=4, num_layers=2)
    model = ScorePureRMTMLP(backbone, head, cfg)
    return model


In [3]:
cfg = env_setting(cfg_name=None)

loader_train, loader_val = build_dataloaders(cfg)

net = build_model(cfg)

focal_loss = FocalLoss()
objective = {'giou': giou_loss, 'l1': l1_loss, 'focal': focal_loss, 'cls': BCEWithLogitsLoss()}
loss_weight = {'giou': cfg.train.GIOU_weight, 'l1': cfg.train.L1_weight, 'focal': 1., 'cls': 1.0}
actor = LeeNetActor(net=net, objective=objective, loss_weight=loss_weight, cfg=cfg)

optimizer, lr_scheduler = get_optimizer_scheduler(net, cfg)

# location loss 没计算出来

trainer = LeeNetTrainer(actor=actor, loaders=[loader_train, loader_val], optimizer=optimizer, lr_scheduler=lr_scheduler, cfg=cfg)

In [4]:
trainer.train(cfg.train.epoch)

out shape in backbone  torch.Size([64, 7680])
gt_boxes_vec shape torch.Size([64, 4])
Epoch Time: 0:00:13.519292
Avg Data Time: 12.89251
Avg GPU Trans Time: 0.04691
Avg Forward Time: 0.57987
out shape in backbone  torch.Size([64, 7680])
gt_boxes_vec shape torch.Size([64, 4])
Epoch Time: 0:00:13.720842
Avg Data Time: 6.46140
Avg GPU Trans Time: 0.04607
Avg Forward Time: 0.35295
out shape in backbone  torch.Size([64, 7680])
gt_boxes_vec shape torch.Size([64, 4])
Epoch Time: 0:00:14.393370
Avg Data Time: 4.47694
Avg GPU Trans Time: 0.04559
Avg Forward Time: 0.27526
out shape in backbone  torch.Size([64, 7680])
gt_boxes_vec shape torch.Size([64, 4])
Epoch Time: 0:00:14.600726
Avg Data Time: 3.36356
Avg GPU Trans Time: 0.04736
Avg Forward Time: 0.23926
out shape in backbone  torch.Size([64, 7680])
gt_boxes_vec shape torch.Size([64, 4])
Epoch Time: 0:00:14.810514
Avg Data Time: 2.69576
Avg GPU Trans Time: 0.04757
Avg Forward Time: 0.21878
out shape in backbone  torch.Size([64, 7680])
gt_boxes

KeyboardInterrupt: 